In [1]:
from PIL import Image, ImageDraw
import random
import os
import shutil
from IPython.core.display import Image as DisplayImage, display

In [2]:
OUTDIR = 'shapes'
NUM_CLASSES = 2
size = (48, 48)

In [3]:
import math

def draw_shape(corners, imgname):
  image = Image.new('RGB', size)
  draw = ImageDraw.Draw(image)

  radius = 15 #random.randint(0, min(size[0], size[1])/2 - 5)
  x0 = random.randint(radius, size[0] - radius)
  y0 = random.randint(radius, size[1] - radius)
  angle = random.uniform(0, math.pi)
  x_prev = math.cos(angle)*radius + x0
  y_prev = math.sin(angle)*radius + y0
  step = math.pi*2/corners
  for i in range(corners):
    angle += step
    x = math.cos(angle)*radius + x0
    y = math.sin(angle)*radius + y0
    #print(x, y)
    draw.line((x_prev, y_prev, x, y), fill=255)
    x_prev = x
    y_prev = y
  del draw  
  image.save(imgname, "PNG")
  del image

In [4]:
def generate_img(count, dir_name):
    full_img_root_dir = os.path.join(OUTDIR, 
                           dir_name)
    # recreate dir
    shutil.rmtree(full_img_root_dir, ignore_errors=True)
    os.makedirs(full_img_root_dir)
    
    for corners in range(3, 3 + NUM_CLASSES):
      dirname = os.path.join(OUTDIR, 
                             dir_name,
                             '{}'.format(corners))
      # Recreate dir
      shutil.rmtree(dirname, ignore_errors=True)
      os.makedirs(dirname)

      for nimage in range(count):
        filename = os.path.join(dirname,
                               '{}.png'.format(nimage))
        draw_shape(corners, filename)
        
        #display(DisplayImage(url=filename))
        #print(filename)

In [5]:
generate_img(1000, 'train')
generate_img(500, 'validate')

In [19]:
generate_img(1000, 'evaluate')

In [6]:
DisplayImage(url= "shapes/train/3/122.png")

In [7]:
DisplayImage(url= "shapes/train/4/133.png")

# Network

In [21]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM, ConvLSTM2D, Reshape, ZeroPadding2D,Convolution2D
from keras.optimizers import SGD
from keras.utils import plot_model

import os
import pickle
import numpy as np
#from IPython.display import Image

In [47]:
batch_size = 100
img_size = size
INPUT_SHAPE = (48, 48, 3)

In [48]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
        os.path.join(OUTDIR, 'train'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 2000 images belonging to 2 classes.


In [49]:
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
        os.path.join(OUTDIR, 'validate'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 1000 images belonging to 2 classes.


In [50]:
evaluation_datagen = ImageDataGenerator()
evaluation_generator = evaluation_datagen.flow_from_directory(
        os.path.join(OUTDIR, 'evaluate'),
        target_size=img_size,
        batch_size=10,
        shuffle=False,
        class_mode='categorical')

Found 2000 images belonging to 2 classes.


In [61]:
model = Sequential()

model.add(ZeroPadding2D((1,1),input_shape=INPUT_SHAPE))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))


In [62]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [63]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=10)

Epoch 1/10
100/100 [==============================] - 175s - loss: 0.8081 - acc: 0.5519 - val_loss: 0.6414 - val_acc: 0.7070
Epoch 2/10
100/100 [==============================] - 174s - loss: 0.5985 - acc: 0.6501 - val_loss: 0.4357 - val_acc: 0.8690
Epoch 3/10
100/100 [==============================] - 176s - loss: 0.4096 - acc: 0.7790 - val_loss: 0.1347 - val_acc: 0.9650
Epoch 4/10
100/100 [==============================] - 177s - loss: 0.2505 - acc: 0.8651 - val_loss: 0.0491 - val_acc: 0.9890
Epoch 5/10
100/100 [==============================] - 178s - loss: 0.1780 - acc: 0.9047 - val_loss: 0.0337 - val_acc: 0.9900
Epoch 6/10
100/100 [==============================] - 178s - loss: 0.1365 - acc: 0.9248 - val_loss: 0.0145 - val_acc: 0.9990
Epoch 7/10
100/100 [==============================] - 178s - loss: 0.1179 - acc: 0.9365 - val_loss: 0.0083 - val_acc: 0.9990
Epoch 8/10
100/100 [==============================] - 178s - loss: 0.1049 - acc: 0.9437 - val_loss: 0.0094 - val_acc: 0.9980


In [64]:
model.save_weights('shapes002.h5')

In [65]:
model.load_weights('shapes002.h5')

In [66]:
model.evaluate_generator(evaluation_generator, 100)

[0.024582737700997656, 0.99499999880790713]